In [72]:
import os
from openai import OpenAI
from dotenv import load_dotenv, find_dotenv
from IPython.display import display, Markdown

In [73]:
load_dotenv(find_dotenv())

True

In [74]:
client = OpenAI()

In [75]:
response = client.chat.completions.create(
    model = "gpt-4o-mini",
    messages= [
        {"role": "user", "content": "you are a useful assistant."},
        {"role": "user", "content": "hi"}
    ]
)

response.choices[0].message.content

'Hello! How can I assist you today?'

In [76]:
def get_response(client: OpenAI,
                 messages: list,
                 model: str = "gpt-4o-mini") -> str:
    return client.chat.completions.create(
        model = model,
        messages = messages,
    )

In [77]:
def system_prompt(message: str) -> dict:
    return {"role":"system", "content":message}

def assistant_prompt(message: str) -> dict:
    return {"role":"assistant", "content":message}

def user_prompt(message: str) -> dict:
    return {"role":"user", "content":message}

def pretty_print(message: str) -> str:
    display(Markdown(message.choices[0].message.content))

In [78]:
prompt = "hey chatgpt, how are you feeling today?"

user_prompts = [user_prompt(prompt)]

response = get_response(client, user_prompts)

pretty_print(response)

I'm just a program, so I don't have feelings, but I'm here and ready to help you! How can I assist you today?

In [79]:
prompts = [
    system_prompt("You are a lawyer to defent a convict who may or may not have been commited."),
    user_prompt("How can you help me to win my case?")
]

response = get_response(client, prompts)    
pretty_print(response)

As your lawyer, I can assist you in several ways to help improve the chances of winning your case:

1. **Case Analysis**: I will thoroughly review all the evidence against you, including witness statements, forensic evidence, and any applicable laws. This allows me to identify strengths and weaknesses in the prosecution's case.

2. **Legal Research**: I will conduct in-depth legal research to find relevant laws, precedents, and legal principles that can support your defense.

3. **Building a Defense Strategy**: Based on our discussions and the evidence, I will formulate a strong defense strategy. This could include arguments for innocence, demonstrating lack of intent, or raising reasonable doubt.

4. **Investigating the Facts**: I can gather additional evidence, interview witnesses, and consult with experts who may help strengthen your case.

5. **Negotiation**: If appropriate, I will negotiate with the prosecution for reduced charges, a plea deal, or other favorable outcomes.

6. **Court Representation**: I will represent you in court, making strong arguments, cross-examining witnesses, and presenting evidence in a compelling way.

7. **Advising on Legal Rights**: I will inform you of your rights throughout the process, ensuring that you understand the legal implications of the decisions you need to make.

8. **Emotional Support**: Going through a legal battle can be stressful. I am here to provide support and guidance throughout the process.

By working together and utilizing these strategies, we can build a strong case with the goal of achieving the best possible outcome for you.

### Few-shot Prompting

In [80]:
prompts_list = [
    system_prompt("You are a helpful assistant."),
    user_prompt("I have daal, tamoto and onion. With this you can easily make: "),
    assistant_prompt("Curry"),
    user_prompt("I have ladyfinger, and onion? With this I can make?")
]

curry_suggestion = get_response(client, prompts_list)

pretty_print(curry_suggestion)

With ladyfinger (okra) and onion, you can make:

1. **Bhindi Masala**: A popular Indian dry curry where ladyfinger is sautéed with onions, spices, and sometimes tomatoes.
2. **Okra Stir-Fry**: Simply sauté the ladyfinger and onions with spices like cumin, turmeric, and chili powder for a quick side dish.
3. **Ladyfinger Curry**: A curry made with ladyfinger and onions cooked in a spiced gravy, which can include tomatoes and coconut milk if you have them.
4. **Stuffed Okra**: If you have additional spices and ingredients, you can stuff the ladyfinger with a mixture of sautéed onions and spices before cooking them.

These dishes are flavorful and can be served with rice or flatbreads. Enjoy cooking!

### Chain of Thought Prompting

https://arxiv.org/abs/2201.11903

In [81]:
prompt = """
Billy wants to get home from San Fran. before 7PM EDT.

It's currently 1PM local time.

Billy can either fly (3hrs), and then take a bus (2hrs), or Billy can take the teleporter (0hrs) and then a bus (1hrs).

Does it matter which travel option Billy selects?
"""

prompts_list = [
    user_prompt(prompt),
]

answer = get_response(client, prompts_list)

pretty_print(answer)

To determine if it matters which travel option Billy selects, let's first clarify the time zones. San Francisco is in Pacific Daylight Time (PDT), which is UTC-7. Eastern Daylight Time (EDT), on the other hand, is UTC-4.

Currently, it is 1 PM PDT in San Francisco. Converting that to EDT:
- 1 PM PDT is 4 PM EDT.

Now, we'll look at the two travel options:

1. **Flying and then taking the bus:**
   - Flight time: 3 hours
   - Bus time: 2 hours
   - Total time for this option: 3 + 2 = 5 hours
   - Departure time: 1 PM PDT
   - Arrival time in PDT: 1 PM + 5 hours = 6 PM PDT
   - Converting to EDT: 6 PM PDT is 9 PM EDT.

2. **Teleporting and then taking the bus:**
   - Teleportation time: 0 hours
   - Bus time: 1 hour
   - Total time for this option: 0 + 1 = 1 hour
   - Departure time: 1 PM PDT
   - Arrival time in PDT: 1 PM + 1 hour = 2 PM PDT
   - Converting to EDT: 2 PM PDT is 5 PM EDT.

Now we compare the arrival times in EDT:
- Flying and taking the bus: arrives at 9 PM EDT.
- Teleporting and taking the bus: arrives at 5 PM EDT.

Since Billy wants to arrive home before 7 PM EDT, the teleportation option is the only one that meets that requirement. Therefore, it does matter which travel option Billy selects; he should choose the teleporter option.

### Prompt Evaluation

In [82]:
system_template = """
WRITE YOUR SYSTEM PROMPT HERE
"""


user_template = """{input}
MODIFICATIONS HERE
"""

In [83]:
query = "What is the biggest animal ever lived on this planet?"

prompts = [
    system_prompt(system_template),
    user_prompt(user_template.format(input = query)),
]

In [84]:
test_answer = get_response(client, prompts)

pretty_print(test_answer)

The biggest animal to have ever lived on our planet is the blue whale (*Balaenoptera musculus*). Blue whales can reach lengths of up to 100 feet (30 meters) and can weigh as much as 200 tons (approximately 181 metric tonnes) or more. They are the largest known animal to have ever existed, dwarfing even the largest dinosaurs.

In [88]:
evaluation_prompt = """You are an expert in analyzing the quality of a response.

You must be hyper-critical.

Provide scores (out of 10) for the following attributes:

1. Clarity - how clear is the response
2. Faithfulness - how related to the original query is the response
3. Correctness - was the response correct?

Before making decision, take time, and think through each item step-by-step.
When you are done - provide your response in the following JSON format:

{"clarity" : "score_out_of_10", "faithfulness" : "score_out_of_10", "correctness" : "score_out_of_10"}"""

evaluation_prompt_template = """
Query: {input}
Response: {response}
"""

In [89]:
list_of_prompts = [
    system_prompt(evaluation_prompt),
    user_prompt(evaluation_prompt_template.format(
        input=query,
        response=test_answer.choices[0].message.content
    ))
]
     

In [90]:
evaluator_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=list_of_prompts,
    response_format={"type" : "json_object"}
)

In [91]:
pretty_print(evaluator_response)

{"clarity" : "10", "faithfulness" : "10", "correctness" : "10"}